<a href="https://colab.research.google.com/github/fundaylncii/MachineLearning/blob/main/K_NearestNeighbors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## gözlemlerin birbirine olan benzerlikleri üzerinden tahmin yapılır.
## knn metodu hem regresyon hem de sınıflandırmaproblemleri için kullanılabilir.

In [ ]:
## Bağımsız değişkenlerin öklid ya da benzeri bir uzaklık hesabı ile her bir gözleme uzaklıkları hesaplanır.
## k : komşu sayısı
## k adet gözlemin bağımlı değişken değerlerinin ortalaması alınır

In [ ]:
## sınıflandırma problemlerinde k adet gözleminin bağımlı değişkenlerinde en sık gözlenen frekansı tahmin edilen sınıf olur.

In [1]:
## Kefişçi Veri Analizi (EDA):

import pandas as pd
from sklearn.metrics import classification_report,roc_auc_score
from sklearn.model_selection import GridSearchCV,cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

pd.set_option("display.max_columns", None)


In [ ]:
## Örnek hastaların diyabet olup olmadığını bulmak için model oluşturmak

In [2]:
df_ = pd.read_csv("/content/diabetes.csv")
df = df_.copy()
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
print("Shape:" ,df.shape)
print("------------------")
print(df.describe().T)
print("------------------")
print(df["Outcome"].value_counts())

Shape: (768, 9)
------------------
                          count        mean         std     min       25%  \
Pregnancies               768.0    3.845052    3.369578   0.000   1.00000   
Glucose                   768.0  120.894531   31.972618   0.000  99.00000   
BloodPressure             768.0   69.105469   19.355807   0.000  62.00000   
SkinThickness             768.0   20.536458   15.952218   0.000   0.00000   
Insulin                   768.0   79.799479  115.244002   0.000   0.00000   
BMI                       768.0   31.992578    7.884160   0.000  27.30000   
DiabetesPedigreeFunction  768.0    0.471876    0.331329   0.078   0.24375   
Age                       768.0   33.240885   11.760232  21.000  24.00000   
Outcome                   768.0    0.348958    0.476951   0.000   0.00000   

                               50%        75%     max  
Pregnancies                 3.0000    6.00000   17.00  
Glucose                   117.0000  140.25000  199.00  
BloodPressure             

In [4]:
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

In [ ]:
## Değişkenlerin standart olması bu gibi işlemlerde modelin daha doğru sonuç vermesini sağlar.

In [5]:
X_scaled = StandardScaler().fit_transform(X)

In [6]:
print(X_scaled) ## columns nameler olmadığı için eklenmeli

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [7]:
X = pd.DataFrame(X_scaled, columns=X.columns)
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496


In [8]:
## Modelleme:

knn_model = KNeighborsClassifier().fit(X,y)

In [9]:
random_user =  X.sample(1, random_state=45)
print(random_user)

     Pregnancies   Glucose  BloodPressure  SkinThickness   Insulin       BMI  \
195     0.342981  1.161295       0.770014       1.283638  1.130518  0.940144   

     DiabetesPedigreeFunction       Age  
195                 -0.232176 -0.360847  


In [10]:
##Tahminleme

knn_model.predict(random_user)

array([1])

In [11]:
## Model Başarı Değerlendirme:
## Confusion matrix için y_pred:
y_pred = knn_model.predict(X)

In [12]:
## auc ve roc hesaplanması için 1 sınıfına ait değerler gerekmektedir.

y_prob = knn_model.predict_proba(X)[:,1]

In [13]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



In [ ]:
## accuracy : başarılı sınıflandırma oranı
## precision : 1 olarak tahmin ettiklerimizin başarı oranı
## recall  : Gerçekte 1 olanları 1 olarak tahmin etme başarı oranımız
## F1-score: precision ve recall harmonik ortalaması
## macro avg : aritmetik ortalama
## weighted avg: ağırlıklı ortalama

In [14]:
## AUC :
roc_auc_score(y, y_prob)

0.9017686567164179

In [16]:
## cross validation:
## 5 katlı validation yapıldı verileri 5 parçaya böldü 4 ü ile model kurdu(5 farklı kombinasyon ile) 1 inde test etti.

cv_resuld = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy","f1","roc_auc"])
print(cv_resuld)

{'fit_time': array([0.01411486, 0.00520205, 0.00570583, 0.00550628, 0.00502753]), 'score_time': array([0.08795142, 0.05452228, 0.03971553, 0.04451466, 0.03440547]), 'test_accuracy': array([0.72077922, 0.73376623, 0.71428571, 0.77124183, 0.7254902 ]), 'test_f1': array([0.58252427, 0.60952381, 0.54166667, 0.63917526, 0.58      ]), 'test_roc_auc': array([0.77555556, 0.78759259, 0.73194444, 0.83226415, 0.77528302])}


In [18]:
print(cv_resuld["test_accuracy"].mean())
print(cv_resuld["test_f1"].mean())
print(cv_resuld["test_roc_auc"].mean())

0.733112638994992
0.5905780011534191
0.7805279524807827


In [ ]:
## Başarı sonuçları arttırma yöntemleri:
## Veri boyutu arttırılabilir.
## Veri önişlemi detaylandırılabilir.
## yeni değişkenler türetilebilir
## İlgili algoritmalar için optimizasyonlar yapılabilir

In [ ]:
## Parametre : modellerin veri içerisinden öğrendiği ağırlıklar
## Hiperparametre : kullanıcı tarafından tanımlanması gereken veri içinden öğrenilemeyen ağırlıklar

In [19]:
## Hiperparametre Optimizasyonu:

knn_model = KNeighborsClassifier()
knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [20]:
knn_params = {"n_neighbors" : range(2,50)}

In [21]:
knn_gs_best = GridSearchCV(knn_model,
             knn_params,
             cv=5,
             n_jobs=-1,
             verbose=1).fit(X,y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [22]:
## optimumum komşuluk sayısı:
knn_gs_best.best_params_

{'n_neighbors': 17}

In [23]:
## Final Model:

knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X,y)

In [25]:
cv_resuld = cross_validate(knn_final,
                           X,
                           y,
                           cv=5,
                           scoring=["accuracy","f1","roc_auc"])
print(cv_resuld["test_accuracy"].mean())
print(cv_resuld["test_f1"].mean())
print(cv_resuld["test_roc_auc"].mean())

0.7669892199303965
0.6170909049720137
0.8127938504542278


In [27]:
random_user =  X.sample(1)
print(random_user)

     Pregnancies   Glucose  BloodPressure  SkinThickness   Insulin       BMI  \
537    -1.141852 -1.999716      -0.470732      -1.288212 -0.692891 -1.306326   

     DiabetesPedigreeFunction       Age  
537                  0.794665  2.872487  


In [28]:
knn_final.predict(random_user)

array([0])

In [29]:
random_user =  X.sample(1)
print(random_user)

     Pregnancies   Glucose  BloodPressure  SkinThickness   Insulin       BMI  \
491    -0.547919 -0.998208         1.0802        0.59363 -0.692891  0.191321   

     DiabetesPedigreeFunction       Age  
491                 -0.543248  0.745293  


In [30]:
knn_final.predict(random_user)

array([0])